# Unidad 4: Sistemas de Recomendación - Learning To Rank - RankNet

- [Github de clase](https://github.com/blancavazquez/CursoDatosMasivosII/blob/master/notebooks/4e_learningToRank.ipynb)


In [2]:
import keras
from keras import backend as K
from keras.layers import Activation, Dense, Input, Subtract
from keras.models import Model

import random
import numpy as np
import pandas as pd 

from sklearn.preprocessing import StandardScaler, MinMaxScaler 

import warnings 
warnings.filterwarnings('ignore')

## Generación y Transformación de Datos

In [6]:
INPUT_DIM = 5

X1 = 2 * np.random.uniform(size=(50, INPUT_DIM))
X2 = np.random.uniform(size=(50, INPUT_DIM))
Y = [random.randint(0,1) for _ in range(50)]

print(X1.shape)
print(X2.shape)
print(len(Y))

(50, 5)
(50, 5)
50


In [7]:
mms = MinMaxScaler()
mms.fit(np.concatenate((X1,X2), axis=0))

X1 = mms.transform(X1)
X2 = mms.transform(X2)
Y = np.asarray(Y)

In [8]:
df_input = pd.DataFrame(X1, columns=["F1", "F2", "F3", "F4", "F5"])
df_input.head()

,F1,F2,F3,F4,F5
0,0.033565,0.040676,0.165044,0.905371,0.444373
1,0.523778,0.449649,0.993151,0.133067,1.000000
2,0.363721,0.661195,0.154168,0.094385,0.542808
3,0.717770,0.522143,0.792144,0.436390,0.677129
4,0.788879,0.425453,0.745835,0.549796,0.888824


## RankNet

In [36]:
def RankNet_model(input_shape):
    """
    Construcción de la red neuronal siamesa
    """
    h1 = Dense(16, activation="relu", name="Relu_layer1")
    h2 = Dense(8, activation="relu", name="Relu_layer2")
    h3 = Dense(4, activation="linear", name="Relu_layer3")
    h4 = Dense(2, activation="linear", name="Relu_layer4")
    h5 = Dense(1, activation="linear", name="Identity_layer")
    
    input1 = Input(shape=(input_shape,), name="Input_layer1")
    x1 = h1(input1)
    x1 = h2(x1)
    x1 = h3(x1)
    x1 = h4(x1)
    x1 = h5(x1)
    
    input2 = Input(shape=(input_shape,), name="Input_layer2")
    x2 = h1(input2)
    x2 = h2(x2)
    x2 = h3(x2)
    x2 = h4(x2)
    x2 = h5(x2)
    
    # Capa de comparación
    subtracted = Subtract(name="Subtract_layer")([x1, x2])
    
    # Función de activación
    out = Activation('sigmoid', name="Activation_layer")(subtracted)
    
    # Modelo
    model = Model(inputs=[input1, input2], outputs=out, name="RankNet_model")
    
    return model

In [37]:
model = RankNet_model(INPUT_DIM)
print(model.summary())

Model: "RankNet_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_layer1 (InputLayer)       [(None, 5)]          0                                            
__________________________________________________________________________________________________
Input_layer2 (InputLayer)       [(None, 5)]          0                                            
__________________________________________________________________________________________________
Relu_layer1 (Dense)             (None, 16)           96          Input_layer1[0][0]               
                                                                 Input_layer2[0][0]               
__________________________________________________________________________________________________
Relu_layer2 (Dense)             (None, 8)            136         Relu_layer1[0][0]    

## Entrenamiento del modelo

In [38]:
model.compile(optimizer='adadelta', loss='binary_crossentropy')
model.fit([X1, X2], Y, batch_size=20, epochs=100)

Epoch 1/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6936
Epoch 2/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6936
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6936
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6936
Epoch 5/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6936
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6936
Epoch 7/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6936
Epoch 8/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6936
Epoch 9/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6936
Epoch 10/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6936
Epoch 11/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6936
Epoch 12/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6936
Epoch 13/100
3/3 [==============================] - 0s 1ms/st

## Salida del modelo

In [39]:
get_ranknet_output = K.function([model.layers[0].input], [model.layers[-3].get_output_at(0)])
Ranknet_output = get_ranknet_output([X1])[0]

In [40]:
df_output = pd.DataFrame(Ranknet_output, columns=["score"])
df_output

,score
0,0.014023
1,-0.055724
2,0.013501
3,-0.031368
4,-0.038032
5,-0.007896
6,0.034615
7,-0.031915
8,-0.006687
9,0.004961
